# Tool1

In [2]:
import requests
from bs4 import BeautifulSoup

In [1]:
from openai import OpenAI
from typing import List, Dict

def ask(messages: List[Dict]):
        client = OpenAI(api_key='YOUR OPENAI KEY')
        response = client.chat.completions.create(
                model = 'gpt-4o-mini-2024-07-18',
                temperature = 0,
                messages = messages,
        )

        return response

## web search tool

In [4]:
## Web Search
from search import internet_search, process_content

class WebSearch:
    def __init__(self, name:str='web_search', threhold:int=8000):
        self.system_prompt = """
You are a Insight Researcher.

1. To find detail informtion for the user query
and summary the content into one sentence as simple as possible
2. If the user's question is about specific numerical values, 
only return the numerical results without any additional explanation.
"""
        self.name = name
        self.description = "the tool use for web search"
        self.threhold = threhold

    def __call__(self, query:str):
        # print(query)
        results = internet_search(query)
        # print(results)
        all_text = ""
        windows_size = 0
        for item in results:
            if windows_size >= self.threhold:
                break
            page_content = process_content(item['href'])
            for page in page_content:
                if windows_size + len(page) > self.threhold:
                    remaining_space = self.threhold - windows_size
                    all_text += page[:remaining_space].strip() + "\n\n"
                    windows_size = self.threhold
                    break
                else:
                    windows_size += len(page)
                    all_text += page + "\n\n"
            if windows_size >= self.threhold:
                break

        msg = [{"role":"system","content":self.system_prompt},
               {"role":"user", "content": f"The search results are {all_text}"}]
        
        answer = ask(messages=msg)

        return answer.choices[0].message.content


In [5]:
search_tool = WebSearch()
result1 = search_tool("population of Toronto")
print(result1)

Toronto's population in 2021 was 2,794,356.


In [6]:
search_tool = WebSearch()
result2 = search_tool("current population of New York City")
print(result2)

The population of New York City in 2024 is approximately 7,931,147.
